In [11]:
from pymongo import MongoClient
import pandas as pd
import re
from datetime import datetime

In [12]:
db = MongoClient('mongodb://10.10.225.142:27017')['cibil']
db.authenticate('nilabja21607', 'pass@123')

True

In [18]:
def multi_chunk_breaker(string: str, splitter: str, shift: int = 2) -> list:
    """ Compute ... and return ...
    :return:
    :param string:
    :param splitter:
    :param shift:
    :return:
    """
    chunk = string.split(splitter)
    chunk.pop(0)
    return [str_breaker(indx[shift:]) for indx in chunk]


def str_breaker(string):
    """
    :param string:
    :return:
    """
    ret = {}
    while string:
        key = string[:2]
        string = string[2:]
        length = int(string[:2])
        string = string[2:]
        val = string[:length]
        string = string[length:]
        ret[key] = val
    return ret


In [19]:
tuef = db['failed'].find_one()

In [20]:
tuef['string']

'TUEF122902680                    0000BP03067777                    100310707446111102019210457PN03N010122EKNATH DHONDIBA SHINDE08012SC10CIBILTUSCR0102010202100308111020190405000-1PA03A010127AT UNCHITHANE POST LADEGAON0222TAL KHATAV DIST SATARA0322TAL KHATAV DIST SATARA0515CITY:PUSESAWALI0706415512080204100830092005PA03A020127AT UNCHITHANE POST LADEGAON0222TAL KHATAV DIST SATARA0311DIST SATARA0515CITY:PUSESAWALI080204100830092004ES0700004490102**'

In [21]:
tuef = tuef['string'][:-17]

In [22]:
keymap = db['keymap'].find_one()

pattern = re.compile(
    "(PN03N01|ID03I01|PT03T01|EC03C01|EM03E01|SC10CIBILTUSCR|PA03A01|"
    "TL04T001|IQ04I001|DR03D01)"
)  # Create the pattern to break the tuef string into segments


In [23]:
chunkdict = {}
breaks = [m.start() for m in pattern.finditer(tuef)]
breaks.insert(0, 0)
breaks.append(len(tuef))

for i in range(1, len(breaks)):
    temp = tuef[breaks[i - 1]:breaks[i]]
    chunkdict[temp[:2]] = temp

header = {}
header["seg_tag"], tuef = tuef[:4], tuef[4:]
header["version"], tuef = tuef[:2], tuef[2:]
header["mem_ref_no"], tuef = tuef[:25].strip(), tuef[25:]
header["blank"], tuef = tuef[:6], tuef[6:]
header["enq_mem_user_id"], tuef = tuef[:30].strip(), tuef[30:]
header["sub_ret_cd"], tuef = tuef[:1], tuef[1:]
header["enq_cntrl_no"], tuef = tuef[:12], tuef[12:]
header["datetime_processed"] = datetime.strptime(
    tuef[:14], '%d%m%Y%H%M%S')
######################################################################
#                                 PN decoding                        #
######################################################################

name_chunk = chunkdict.get('PN')
if name_chunk:
    name = str_breaker(name_chunk[7:])
    name = {keymap['PN'][key]: val for key, val in name.items()}
    if 'gender' in name:
        name['gender'] = keymap['gender'][name['gender']]
    if 'dob' in name:
        name['dob'] = datetime.strptime(name['dob'], '%d%m%Y')
    name = {key: val for key, val in name.items() if val is not None}
    header.update(name)
######################################################################
#                                 ID decoding                        #
######################################################################

id_chunk = chunkdict.get('ID')
if id_chunk:
    ids = pd.DataFrame(multi_chunk_breaker(id_chunk, "ID03I"))
    ids.columns = [keymap['ID'][key] for key in ids.columns]
    if "id_type" in ids:
        ids["id_type"] = [keymap["id_type"][i] for i in
                          ids["id_type"]]
    if "issue_date" in ids:
        ids["issue_date"] = pd.to_datetime(ids["issue_date"],
                                           format='%d%m%Y')
    if "expiration_date" in ids:
        ids["expiration_date"] = pd.to_datetime(
            ids["expiration_date"], format='%d%m%Y')
    header['ids'] = ids.T.apply(lambda x: x.dropna().to_dict()).\
        tolist()
######################################################################
#                            PT decoding                             #
######################################################################

phone_chunk = chunkdict.get('PT')
if phone_chunk:
    tele = pd.DataFrame(multi_chunk_breaker(phone_chunk, "PT03T"))
    tele.columns = [keymap['PT'][key] for key in tele.columns]
    if "telephone_type" in tele:
        tele["telephone_type"] = [keymap["telephone_type"][i] for i in
                                  tele["telephone_type"]]
    header['phones'] = tele.T.apply(
        lambda x: x.dropna().to_dict()).tolist()
######################################################################
#                                 EC decoding                        #
######################################################################

email_chunk = chunkdict.get('EC')
if email_chunk:
    email = pd.DataFrame(multi_chunk_breaker(email_chunk, "EC03C"))
    email.columns = ['email']
    header['emails'] = \
        email.email.str.lower().dropna().unique().tolist()

######################################################################
#                            EM decoding                             #
######################################################################

employ_chunk = chunkdict.get('EM')
if employ_chunk:
    employ = str_breaker(employ_chunk[7:])
    employ = {keymap['EM'][key]: val for key, val in employ.items()}
    employ['account_type'] = keymap['account_type'][
        employ['account_type']]
    employ['date_reported_and_certified'] = datetime.strptime(
        employ['date_reported_and_certified'], '%d%m%Y')
    employ['occupation_code'] = keymap['occupation_code'].get(
        employ.get('occupation_code'))
    employ['net_gross_income_indicator'] = \
        keymap['net_gross_income_indicator'].get(employ.get(
            'net_gross_income_indicator'))
    employ['monthly_annual_income_indicator'] = \
        keymap['monthly_annual_income_indicator'].get(
            employ.get('monthly_annual_income_indicator'))
    employ = {key: val for key, val in employ.items()
              if val is not None}
    header.update(employ)
######################################################################
#                       SC decoding                                  #
######################################################################

score_chunk = chunkdict.get("SC")
if score_chunk:
    score = str_breaker(score_chunk[14:])
    score = {keymap['SC'][key]: val for key, val in score.items()}
    score['score_date'] = datetime.strptime(score['score_date'],
                                            '%d%m%Y')
    score = {key: val for key, val in score.items()
             if val is not None}
    header.update(score)
######################################################################
#                                 PA decoding                        #
######################################################################

addr_chunk = chunkdict.get("PA")
if addr_chunk:
    addr = pd.DataFrame(multi_chunk_breaker(addr_chunk, "PA03A"))
    addr.columns = [keymap['PA'][key] for key in addr.columns]
    if 'date_reported' in addr:
        addr['date_reported'] = pd.to_datetime(addr['date_reported'],
                                               format='%d%m%Y')
    addr['state_code'] = [keymap['state_code'].get(i) for i in
                          addr['state_code']]
    addr['address_category'] = [keymap['address_category'].get(i)
                                for i in addr['address_category']]
    header['addresses'] = addr.T.apply(
        lambda x: x.dropna().to_dict()).tolist()

######################################################################
#                       TL decoding                                  #
######################################################################

acc_chunk = chunkdict.get('TL')
if acc_chunk:
    acc = multi_chunk_breaker(acc_chunk, "TL04T", 3)
    summary = pd.DataFrame([i for i in acc if
                            i['02'] == "ACCTREVIEW_SUMM"])
    acc = pd.DataFrame([i for i in acc if i['02']
                        != "ACCTREVIEW_SUMM"])
    acc.columns = [keymap['TL'][key] for key in acc.columns]
    summary.columns = [keymap['summary'][key] for
                       key in summary.columns]
    acc['account_type'] = [keymap['account_type'][i] for i in
                           acc['account_type']]
    acc['ownership_indicator'] = [keymap['ownership_indicator'][i] for
                                  i in acc['ownership_indicator']]
    datecols = list({'date_opened_disbursed', 'date_of_last_payment',
                     'date_closed', 'date_reported_certified',
                     'payment_history_start_date',
                     'payment_history_end_date'}.intersection(
        acc.columns))
    acc[datecols] = acc[datecols].apply(pd.to_datetime,
                                        format="%d%m%Y")
    if 'suit_filed_wilful_default' in acc:
        acc['suit_filed_wilful_default'] = \
            [keymap['suit_filed_wilful_default'].get(i) for i in
             acc['suit_filed_wilful_default']]
    header['accounts'] = acc.T.apply(
        lambda x: x.dropna().to_dict()).tolist()
######################################################################
#                                 IQ decoding                        #
######################################################################
enquiry_chunk = chunkdict.get('IQ')
if enquiry_chunk:
    enq = pd.DataFrame(multi_chunk_breaker(enquiry_chunk, "IQ04I", 3))
    enq.columns = [keymap['IQ'][key] for key in enq.columns]
    enq['date_of_enquiry'] = pd.to_datetime(enq['date_of_enquiry'],
                                            format='%d%m%Y')
    enq['enquiry_purpose'] = [keymap['account_type'][i] for i in
                              enq['enquiry_purpose']]
    header['enquiries'] = enq.T.apply(
        lambda x: x.dropna().to_dict()).tolist()


KeyError: 'state_code'

In [27]:
addr_chunk = chunkdict.get("PA")
# if addr_chunk:
addr = pd.DataFrame(multi_chunk_breaker(addr_chunk, "PA03A"))
addr.columns = [keymap['PA'][key] for key in addr.columns]
#     if 'date_reported' in addr:
#         addr['date_reported'] = pd.to_datetime(addr['date_reported'],
#                                                format='%d%m%Y')
#     addr['state_code'] = [keymap['state_code'].get(i) for i in
#                           addr['state_code']]
#     addr['address_category'] = [keymap['address_category'].get(i)
#                                 for i in addr['address_category']]
#     header['addresses'] = addr.T.apply(
#         lambda x: x.dropna().to_dict()).tolist()
addr

,address_line1,address_line2,address_line3,address_line5,pin_code,address_category,date_reported
0,AT UNCHITHANE POST LADEGAON,TAL KHATAV DIST SATARA,TAL KHATAV DIST SATARA,CITY:PUSESAWALI,415512,04,30092005
1,AT UNCHITHANE POST LADEGAON,TAL KHATAV DIST SATARA,DIST SATARA,CITY:PUSESAWALI,NaN,04,30092004


In [14]:
tuef

'04102019110620PN03N010115THIPPESWAMY C B08012SC10CIBILTUSCR0102010202100308041020190405000-1PA03A010132KONDLAHALLI CHITRADURGA DISTRICT0602290706577521080204100830092004'

In [15]:
chunkdict

{'TU': 'TUEF122886032                    0000BP03067777                    100309654099704102019110620',
 'PN': 'PN03N010115THIPPESWAMY C B08012',
 'SC': 'SC10CIBILTUSCR0102010202100308041020190405000-1',
 'PA': 'PA03A010132KONDLAHALLI CHITRADURGA DISTRICT0602290706577521080204100830092004'}

In [14]:
db['raw_tuef'].drop()
db['tuef'].drop()
db['failed'].drop()

In [5]:
keymap = pd.read_excel('data/keyword_mapper.xlsx', sheet_name=None, dtype=str)
# for key, val in keymap.items():
#     if key != "reason_code":
#         keymap[key] = {i['Field_tag']: i['Name'] for i in
#                        val.to_dict(orient='record')}

In [2]:
import pandas as pd

{'01': {'Name': 'Auto Loan (Personal) ', 'loan_type': ' Secured '},
 '02': {'Name': 'Housing Loan ', 'loan_type': ' Secured '},
 '03': {'Name': 'Property Loan ', 'loan_type': ' Secured '},
 '04': {'Name': 'Loan Against Shares/Securities ', 'loan_type': ' Secured '},
 '05': {'Name': 'Personal Loan ', 'loan_type': ' Unsecured '},
 '06': {'Name': 'Consumer Loan ', 'loan_type': ' Unsecured '},
 '07': {'Name': 'Gold Loan ', 'loan_type': ' Secured '},
 '08': {'Name': 'Education Loan ', 'loan_type': ' Unsecured '},
 '09': {'Name': 'Loan to Professional ', 'loan_type': ' Unsecured '},
 '10': {'Name': 'Credit Card ', 'loan_type': ' Unsecured '},
 '11': {'Name': 'Leasing ', 'loan_type': ' Secured '},
 '12': {'Name': 'Overdraft ', 'loan_type': ' Unsecured '},
 '13': {'Name': 'Two-wheeler Loan ', 'loan_type': ' Secured '},
 '14': {'Name': 'Non-Funded Credit Facility ', 'loan_type': ' Secured '},
 '15': {'Name': 'Loan Against Bank Deposits ', 'loan_type': ' Secured '},
 '16': {'Name': 'Fleet Card '